# Cuaderno para consumir de Trino

En este cuaderno se muestra un ejemplo para consumir del servicio de Trino una vez  que los datos esten cargados a MinIO y las tbalas definidas en Hive-Metastore.


## Librerias

In [1]:
import trino
import pandas as pd

## Definición de la coexión

In [2]:
conn=trino.dbapi.connect(
    host='trino',
    port=8080,
    user='jupyter',
    catalog='hive',
    schema='default',
)

## Ejemplo 

Podemos definir las consultas como sql y obtener las tablas.
En este ejemplo se seleccionan todos los datos de una tabla

In [3]:
cur = conn.cursor()
cur.execute('SELECT * FROM conjunto_de_datos_erogaciones_enigh_2018_ns')
rows = cur.fetchall()

# El nombre de las columnas no se regresa al hacer la consulta por lo que se hace una consulta especial para estos 
cur = conn.cursor()
cur.execute("SELECT column_name FROM information_schema.columns WHERE table_name = 'conjunto_de_datos_erogaciones_enigh_2018_ns'")
columns = cur.fetchall()

pd.DataFrame(rows,columns=[x[0] for x in columns])

,folioviv,foliohog,clave,mes_1,mes_2,mes_3,mes_4,mes_5,mes_6,ero_1,ero_2,ero_3,ero_4,ero_5,ero_6,ero_tri
0,100013602,1,Q003,9,8,7,6,5,4,3000,3000,3000,3000,3000,3000,8852
1,100013602,1,Q001,9,8,7,6,5,4,20,0,0,0,0,0,10
2,100013603,1,Q004,10,9,8,7,6,5,5000,5000,5000,5000,5000,5000,14754
3,100013603,1,Q001,10,9,8,7,6,5,15000,20000,30000,20000,30000,20000,66393
4,100026701,1,Q003,9,8,7,6,5,4,5000,5000,5000,5000,5000,5000,14754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48498,3202290421,1,Q100,8,,,,,,1000,None,None,None,None,None,2903
48499,3202455008,1,Q100,10,,,,,,1350,None,None,None,None,None,3919
48500,3202455724,1,Q100,9,,,,,,1800,None,None,None,None,None,5400
48501,3260583420,1,Q100,8,,,,,,1500,None,None,None,None,None,4355


Podemos tambien obterner todos los nombres de las tablas definidas en el schema

In [4]:
cur.execute("SHOW TABLES")
tables = cur.fetchall()
tables

[['conjunto_de_datos_agro_enigh_2018_ns'],
 ['conjunto_de_datos_concentradohogar_enigh_2018_ns'],
 ['conjunto_de_datos_erogaciones_enigh_2018_ns'],
 ['conjunto_de_datos_gastoshogar_enigh_2018_ns'],
 ['conjunto_de_datos_gastospersona_enigh_2018_ns'],
 ['conjunto_de_datos_gastotarjetas_enigh_2018_ns'],
 ['conjunto_de_datos_hogares_enigh_2018_ns'],
 ['conjunto_de_datos_ingresos_enigh_2018_ns'],
 ['conjunto_de_datos_noagro_enigh_2018_ns'],
 ['conjunto_de_datos_poblacion_enigh_2018_ns'],
 ['conjunto_de_datos_trabajos_enigh_2018_ns'],
 ['conjunto_de_datos_vivienda_enigh_2018_ns']]

En este ejemplo hacemos la consulta de la poblacion total para 2018. La cual coincide con la tabla oficial publicada en 

https://www.inegi.org.mx/contenidos/programas/enigh/nc/2018/tabulados/enigh2018_ns_basicos_tabulados.xlsx

In [5]:
cur.execute(""" 
SELECT SUM(INTEGRANTES*FACTOR) AS POBLACION
FROM conjunto_de_datos_vivienda_enigh_2018_ns AS VIV
INNER JOIN (
    SELECT FOLIOVIV, COUNT(*) AS INTEGRANTES
    FROM conjunto_de_datos_poblacion_enigh_2018_ns 
    GROUP BY FOLIOVIV
) POB 
ON VIV.FOLIOVIV = POB.FOLIOVIV
""")
RES = cur.fetchall()

"{:,}".format(int(RES[0][0]))

'125,189,618'

LA información anterior puede ser desglosada por grupos de edad 

In [6]:
cur.execute(""" 
SELECT G_EDAD, SUM(INTEGRANTES*FACTOR) AS POBLACION
FROM conjunto_de_datos_vivienda_enigh_2018_ns AS VIV
INNER JOIN (
    SELECT FOLIOVIV, G_EDAD, COUNT(*) AS INTEGRANTES
    FROM (SELECT FOLIOVIV, CASE 
            WHEN EDAD < 5 THEN '00-04 AÑOS'
            WHEN EDAD BETWEEN 5 AND 11 THEN '05-11 AÑOS'
            WHEN EDAD BETWEEN 12 AND 19 THEN '12-19 AÑOS'
            WHEN EDAD BETWEEN 20 AND 29 THEN '20-29 AÑOS'
            WHEN EDAD BETWEEN 30 AND 39 THEN '30-39 AÑOS'
            WHEN EDAD BETWEEN 40 AND 49 THEN '40-49 AÑOS'
            WHEN EDAD BETWEEN 50 AND 59 THEN '50-59 AÑOS'
            WHEN EDAD > 59  THEN '60 Y MÁS AÑOS'
            END AS G_EDAD
        FROM conjunto_de_datos_poblacion_enigh_2018_ns 
    )
    GROUP BY FOLIOVIV, G_EDAD
) POB 
ON VIV.FOLIOVIV = POB.FOLIOVIV
GROUP BY G_EDAD
""")
RES2 = cur.fetchall()

In [ ]:
datos = pd.DataFrame(RES2, columns = ["G_EDAD","POBLACION"])
datos["formato"] = datos["POBLACION"].astype(int).apply(lambda x: "{:,}".format(x))
datos.sort_values("G_EDAD")